In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.applications import MobileNetV3Small
import time
keras = tf.keras
layers = tf.keras.layers
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0  # normalize to [0,1] range
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\0\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\0\test'
checkpoint_path = r'C:\Users\24306\Desktop\MobileNetV3Large\0\MobileNetV3Large.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[86.0, 40.0, 16.0, 37.0, 96.0, 52.0, 13.0, 86.0, 95.0, 55.0, 95.0, 13.0, 52.0, 16.0, 16.0, 37.0, 80.0, 93.0, 80.0, 13.0, 96.0, 96.0, 20.0, 7.0, 45.0, 16.0, 37.0, 66.0, 23.0, 23.0, 45.0, 45.0, 96.0, 39.0, 86.0, 96.0, 32.0, 95.0, 40.0, 84.0, 16.0, 86.0, 80.0, 93.0, 100.0, 45.0, 96.0, 35.0, 40.0, 100.0, 40.0, 10.0, 10.0, 23.0, 84.0, 86.0, 95.0, 93.0, 13.0, 66.0, 100.0, 20.0, 7.0, 37.0, 84.0, 23.0, 100.0, 93.0, 16.0, 96.0, 7.0, 13.0, 37.0, 47.0, 80.0, 80.0, 4.0, 7.0, 40.0, 37.0, 47.0, 100.0, 7.0, 47.0, 23.0, 84.0, 13.0, 93.0, 13.0, 86.0, 37.0, 93.0, 32.0, 66.0, 96.0, 100.0, 66.0, 86.0, 23.0, 37.0, 80.0, 95.0, 16.0, 20.0, 37.0, 86.0, 47.0, 80.0, 37.0, 100.0, 100.0, 16.0, 96.0, 13.0, 100.0, 40.0, 80.0, 16.0, 80.0, 16.0, 37.0, 16.0, 55.0, 45.0, 7.0, 32.0, 86.0, 93.0, 37.0, 84.0, 86.0, 13.0, 23.0, 47.0, 13.0, 13.0, 100.0, 40.0, 35.0, 80.0, 47.0, 40.0, 40.0, 4.0, 86.0, 93.0, 32.0, 10.0, 40.0, 95.0, 80.0, 52.0, 16.0, 55.0, 45.0, 80.0, 95.0, 84.0, 47.0, 16.0, 47.0, 86.0, 40.0, 20.0, 37.0, 10.0, 3

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
MobileNetV3Large = tf.keras.applications.MobileNetV3Large(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
MobileNetV3Large.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
x = MobileNetV3Large(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

12683000/12683000 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 961       
                                                                 
Total params: 2,997,313
Trainable params: 2,972,913
Non-trainable params: 24,400
_______________________________________________

In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
np.savetxt('MobileNetV3Large_loss.txt',history.history['loss'])
#np.savetxt('DenseNet201_acc.txt',history.history['acc'])
np.savetxt('MobileNetV3Large_val_loss.txt',history.history['val_loss'])
#np.savetxt('DenseNet201_val_acc.txt',history.history['val_acc'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Tue Apr 25 17:02:12 2023
Epoch 1/300
590/591 [============================>.] - ETA: 0s - loss: 294.7225

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\0\MobileNetV3Large.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\0\MobileNetV3Large.ckpt\assets


591/591 [==============================] - 64s 91ms/step - loss: 294.2380 - val_loss: 3818.5913
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 19.5970

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\0\MobileNetV3Large.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\0\MobileNetV3Large.ckpt\assets


591/591 [==============================] - 85s 144ms/step - loss: 0.2008 - val_loss: 0.1961
Epoch 273/300
591/591 [==============================] - 65s 110ms/step - loss: 0.2345 - val_loss: 0.2121
Epoch 274/300
591/591 [==============================] - 65s 109ms/step - loss: 0.3133 - val_loss: 0.2215
Epoch 275/300
591/591 [==============================] - 65s 110ms/step - loss: 0.2280 - val_loss: 0.2058
Epoch 276/300
591/591 [==============================] - 65s 111ms/step - loss: 0.2403 - val_loss: 0.2387
Epoch 277/300
591/591 [==============================] - 63s 107ms/step - loss: 0.2472 - val_loss: 0.2007
Epoch 278/300
591/591 [==============================] - 63s 106ms/step - loss: 0.2470 - val_loss: 0.2295
Epoch 279/300
591/591 [==============================] - 63s 107ms/step - loss: 0.2136 - val_loss: 0.2196
Epoch 280/300
591/591 [==============================] - 63s 107ms/step - loss: 0.2011 - val_loss: 0.2234
Epoch 281/300
591/591 [==============================] - ETA

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\0\MobileNetV3Large.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\0\MobileNetV3Large.ckpt\assets


591/591 [==============================] - 86s 145ms/step - loss: 0.2176 - val_loss: 0.1897
Epoch 282/300
591/591 [==============================] - 62s 104ms/step - loss: 0.2120 - val_loss: 0.2087
Epoch 283/300
591/591 [==============================] - 61s 103ms/step - loss: 0.2017 - val_loss: 0.2255
Epoch 284/300
591/591 [==============================] - 62s 104ms/step - loss: 0.2383 - val_loss: 0.2129
Epoch 285/300
591/591 [==============================] - 61s 103ms/step - loss: 0.2343 - val_loss: 0.2180
Epoch 286/300
591/591 [==============================] - 61s 104ms/step - loss: 0.2755 - val_loss: 0.2076
Epoch 287/300
591/591 [==============================] - 61s 103ms/step - loss: 0.2119 - val_loss: 0.2369
Epoch 288/300
591/591 [==============================] - 61s 104ms/step - loss: 0.2243 - val_loss: 0.2436
Epoch 289/300
591/591 [==============================] - 61s 104ms/step - loss: 0.2589 - val_loss: 0.2482
Epoch 290/300
591/591 [==============================] - 62s